Se quiere modelar y resolver el problema de flujo óptimo para el sistema representado por el unifilar de la figura. Se utilizará el modelo DC-OPF.



In [0]:
from IPython.display import Image
from IPython.core.display import HTML 


Image(url= "https://drive.google.com/uc?id=1zKqS2zehsZ2IxdM8sdNr7CHbdVfmTxtW", width=750)



El modelo DC-OPF Es dado por las ecuaciones 

$$\bf p_L= H\mathbf \theta = X^{-1} A \theta$$

para el vector de potencias $\bf p_L$  por las $M=6$ líneas. En estas ecuaciones $X$ es  una matriz diagonal de dimensiones $M\times M$ con las reactancias de las líneas en la diagonal, $\theta$ es el vector de ángulos en las $N=5$ barras, y $A$ es la matriz de incidencia de dimensiones $M\times  N$.

También se tiene para el vector $\bf p_B$ de potencias  inyectadas en las barras

$$\bf p_B = B\theta = A^T X^{-1} A \theta$$

Para la red de la figura, con la numeración y los sentidos de las líneas  elegidos según las flechas rojas del esquema mostrado más abajo, se tiene  (A COMPLETAR)

$$
A=\begin{bmatrix}
    0  & 0 & 0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 \\
\end{bmatrix}
$$

$$
X=\begin{bmatrix}
    0  & 0 & 0 & 0 & 0 & 0\\
   0  & 0 & 0 & 0 & 0 & 0\\
   0  & 0 & 0 &  0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 & 0 \\
   0  & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}
$$

con $x_{12}=0.0281$, $x_{23}=0.0108$, $x_{34}=0.0297$, $x_{45}=0.0297$, $x_{14}=0.0304$, $x_{15}=0.0064$ (en p.u.).





In [0]:
Image(url= "https://drive.google.com/uc?id=1uoDeKu9d-sZhFM3R-2gmdplqNirh0cFf", width=750)


Luego el flujo de carga óptimo para esta red puede obtenerse con el siguiente codigo en CVXPY


In [0]:
# Import packages.
import cvxpy as cp
import numpy as np



In [0]:
# Modelo de cinco barras a modificar
Sbase = 100 # MVA
M=6
N=5
A=np.array([ [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0] ]) 
x54 = 0.0297; # pu
x51 = 0.0064;
x14 = 0.0304;
x43 = 0.0297;
x12 = 0.0281;
x32 = 0.0108;

#print(A)

X=np.array([ [0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0] ])
#print(X)
H=np.linalg.inv(X)@A
B= A.transpose()@H
#print(H)
#print(B)

# pasar a pu.
d = np.divide(np.array([0,300,300,400,0]),Sbase)
plmax = np.divide(np.array([240,400,400,400,400,400]),Sbase)
gmax = np.divide(np.array([40,170,520,200,600]),Sbase)

c = np.array([14,15,30,40,10])

G2pg=np.array([[1,1,0,0,0],[0,0,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]])

pb = cp.Variable(5) # potencia neta inyectada en las barras = pg-d
g = cp.Variable(5) # potencia de cada generador 
pl = cp.Variable(6) # potencia por las líneas
pg = cp.Variable(5) # potencias inyectadas por los generadores en las barras; pg = G2pg*g
theta = cp.Variable(5) # ángulos de las barras

cost = cp.sum(c*g)

# a continuación van las restricciones; ya hay algunas ingresadas; hay que agregar las restricciones en las líneas y los generadores.
constraints = [ theta[0]==0 , G2pg*g == pg , pb==B*theta , pl==H*theta , pb==pg-d]
 

prob = cp.Problem(cp.Minimize(cost),constraints)
prob.solve()

# recuperamos los valores óptimos; hay que convertirlos usando la potencia base
potencia_real_generadores = np.multiply(g.value,Sbase)
potencia_real_en_barra = np.multiply(pg.value,Sbase)
potencia_real_en_linea = np.multiply(pl.value,Sbase)

print("El costo óptimo es", prob.value*Sbase)
print("La generación óptima es, %s" %potencia_real_generadores)
print("En cada barra, se genera %s" %potencia_real_en_barra)
print("La potencia en cada línea es, %s" %potencia_real_en_linea)

#print(theta.value)
#print(H@theta.value-pl.value)
#print(B@theta.value-pb.value)
#print(pb.value-pg.value+d)
#print(prob.status)

print("El vector multiplicador de Lagrange de la restricción de abastecimiento de la demanda en cada nodo es lambda_n= %s" %constraints[4].dual_value)
print("El vector multiplicador de Lagrange de la restricción de límite en las líneas es es lambda_l= %s" %constraints[3].dual_value)
print("El vector multiplicador de Lagrange de la restricción de límite de generación es lambda_g= %s" %constraints[6].dual_value)


El costo óptimo es 17482.418573738698
La generación óptima es, [ 40.00234897 170.00234897 276.17425148  31.63069081 482.18952768]
En cada barra, se genera [2.10004700e+02 4.93462201e-07 2.76174251e+02 3.16306889e+01
 4.82189530e+02]
La potencia en cada línea es, [239.98842667 242.20150143 183.4724204   55.09122845 268.73445699
  31.26549547]
El vector multiplicador de Lagrange de la restricción de abastecimiento de la demanda en cada nodo es lambda_n= [16.95513025 26.39439539 30.01633506 40.01001434  9.96868341]
El vector multiplicador de Lagrange de la restricción de límite en las líneas es es lambda_l= [-6.25582286e+01  2.05611863e-02  5.97770732e-03  6.63693981e-03
  9.75469475e-03 -9.71199556e-03]
El vector multiplicador de Lagrange de la restricción de límite de generación es lambda_g= [ 2.95407528e+00  1.95407528e+00  0.00000000e+00  0.00000000e+00
 -4.16333634e-16]


a) Armar el problema completo y correrlo.

b) Obtener los multiplicadores de Lagrange de las distintas restricciones e interpretarlos.

c) Reflexionar sobre qué sucedería si se modificaran los costos de generación y luego correr el problema con nuevos costos.

d) Reflexionar sobre qué sucedería si se modificaran los límites de las líneas y luego correr el problema con nuevos costos.



In [0]:
#Escribir código aquí